# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using Turi Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Turi Create

In [1]:
import turicreate

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = turicreate.SFrame('home_data.sframe/')

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
                'bathrooms',
                'sqft_living', 'sqft_living_sqrt',
                'sqft_lot', 'sqft_lot_sqrt',
                'floors', 'floors_square',
                'waterfront', 'view', 'condition', 'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in Turi Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = turicreate.linear_regression.create(sales, target='price', features=all_features,
                                                validation_set=None, 
                                                l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.347050     | 6869667.895833     | 385998.458623                   |

| 10        | 11       | 0.000002  | 1.447531     | 6842123.232651     | 364204.576180                   |

+-----------+----------+-----------+--------------+--------------------+---------------------------------+

Completed (Iteration limit reached).

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [6]:
model_all.coefficients[model_all.coefficients['value']==0].print_rows(num_rows=len(model_all.coefficients))

+-----------------+-------+-------+--------+
|       name      | index | value | stderr |
+-----------------+-------+-------+--------+
|     bedrooms    |  None |  0.0  |  None  |
| bedrooms_square |  None |  0.0  |  None  |
|     sqft_lot    |  None |  0.0  |  None  |
|  sqft_lot_sqrt  |  None |  0.0  |  None  |
|      floors     |  None |  0.0  |  None  |
|  floors_square  |  None |  0.0  |  None  |
|    waterfront   |  None |  0.0  |  None  |
|       view      |  None |  0.0  |  None  |
|    condition    |  None |  0.0  |  None  |
|  sqft_basement  |  None |  0.0  |  None  |
|     yr_built    |  None |  0.0  |  None  |
|   yr_renovated  |  None |  0.0  |  None  |
+-----------------+-------+-------+--------+
[12 rows x 4 columns]



In [7]:
model_all.coefficients[model_all.coefficients['value']!=0][:]

name,index,value,stderr
(intercept),None,274873.05595049576,None
bathrooms,None,8468.531086910105,None
sqft_living,None,24.42072098244546,None
sqft_living_sqrt,None,350.06055338605546,None
grade,None,842.0680348976231,None
sqft_above,None,20.024722417091112,None


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [8]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [9]:
import numpy as np

In [10]:
l1_search = np.logspace(1, 7, num=13)

In [11]:
def model_l1(train_data=training, valid_data=validation, l1_penalty=10., 
             output_name='price', features_list=all_features):
    model = turicreate.linear_regression.create(train_data, target=output_name, features=features_list,
                                                validation_set=None, 
                                                l2_penalty=0., l1_penalty=l1_penalty, verbose=False)
    e = compute_rss(model.predict(valid_data[features_list]), valid_data[output_name])
    print('L1', l1_penalty, ':', e)
    return e

In [12]:
def compute_rss(pred, actual):
    return sum((pred-actual)**2)

In [13]:
model_list = []
for l1 in l1_search:
    model_list.append(model_l1(l1_penalty = l1))

L1 10.0 : 625766285142461.2
L1 31.622776601683793 : 625766285362395.4
L1 100.0 : 625766286057886.9
L1 316.22776601683796 : 625766288257224.8
L1 1000.0 : 625766295212185.9
L1 3162.2776601683795 : 625766317206077.6
L1 10000.0 : 625766386760661.5
L1 31622.776601683792 : 625766606749281.1
L1 100000.0 : 625767302791633.1
L1 316227.7660168379 : 625769507643885.0
L1 1000000.0 : 625776517727025.6
L1 3162277.6601683795 : 625799062845466.8
L1 10000000.0 : 625883719085424.4


*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

In [14]:
min(model_list)

625766285142461.2

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [15]:
model_l1_10 = turicreate.linear_regression.create(sales, target='price', features=all_features,
                                                validation_set=None, 
                                                l2_penalty=0., l1_penalty=10,verbose=False)

model_l1_10.coefficients[model_l1_10.coefficients['value']!=0].print_rows(num_rows=len(model_l1_10.coefficients))

+------------------+-------+----------------------+--------+
|       name       | index |        value         | stderr |
+------------------+-------+----------------------+--------+
|   (intercept)    |  None |  20459.247521910453  |  None  |
|     bedrooms     |  None |  8155.3809873690225  |  None  |
| bedrooms_square  |  None |  1479.7378742257197  |  None  |
|    bathrooms     |  None |  24576.238317244497  |  None  |
|   sqft_living    |  None |  37.491150479793994  |  None  |
| sqft_living_sqrt |  None |  1109.395970730788   |  None  |
|     sqft_lot     |  None | -0.01684991984605998 |  None  |
|  sqft_lot_sqrt   |  None |  149.56942398529617  |  None  |
|      floors      |  None |   20983.5137367872   |  None  |
|  floors_square   |  None |  12278.102345064515  |  None  |
|    waterfront    |  None |  581971.3066492834   |  None  |
|       view       |  None |  92988.98996857251   |  None  |
|    condition     |  None |  6924.287196570817   |  None  |
|      grade       |  No

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [16]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [17]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model.coefficients['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [18]:
def model_l1_limit(train_data=training, valid_data=validation, l1_penalty=10., 
             output_name='price', features_list=all_features):
    model = turicreate.linear_regression.create(train_data, target=output_name, features=features_list,
                                                validation_set=None, 
                                                l2_penalty=0., l1_penalty=l1_penalty, verbose=False)
    e = compute_rss(model.predict(valid_data[features_list]), valid_data[output_name])
    nnzz = [i for i in list(model.coefficients['value']) if i != 0]
    print('L1', l1_penalty, ':', len(nnzz), e)
    return e, len(nnzz)

In [19]:
model_list2 = []
non_zeros = []
#list2_coeffs = []
for l1_pen in l1_penalty_values:
    rss, nzs= model_l1_limit(l1_penalty=l1_pen)
    model_list2.append(rss)
    non_zeros.append(nzs)
    #list2_coeffs.append(coefficients)

L1 100000000.0 : [25090.917367212027, 7789.1770611005595, 847.5596869428736, 25234.209194458625, 39.03944596356861, 1117.3118955675634, -0.025686118239919474, 143.98899196976998, 20695.35923964387, 12466.690650291852, 568204.6445841155, 91066.94280879851, 6360.78092624981, 6139.2128056478905, 43.03582992460563, 118.9458749543526, 9.040401654020457, 48.615467309274564] 627492659875075.4
L1 127427498.57031322 : [26746.66193663547, 7743.979047849886, 822.358945250938, 25178.62593058694, 39.01071813525402, 1114.9107159232628, -0.01866307372278282, 142.51979784137163, 20545.867304697575, 12339.245250245609, 558930.2470718037, 90439.72185124883, 6288.009465538855, 6118.412320616765, 42.957556768130495, 117.99004260104358, 8.924085433195666, 46.56185624767167] 628210516770747.1
L1 162377673.91887242 : [28873.181016622366, 7691.047075686324, 790.9175796843566, 25115.278534515925, 38.9820788132087, 1112.2394146479883, -0.024737360580801046, 140.94584475081723, 20365.265896852678, 12181.18625770

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [20]:
l1_penalty_min = np.max(l1_penalty_values[[i for i in range(len(non_zeros)) if non_zeros[i] > max_nonzeros]])
l1_penalty_min

2976351441.6313133

In [21]:
l1_penalty_max = np.min(l1_penalty_values[[i for i in range(len(non_zeros)) if non_zeros[i] < max_nonzeros]])
l1_penalty_max

3792690190.7322536

***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [22]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [23]:
model_list2 = []
non_zeros = []
#list2_coeffs = []
for l1_pen in l1_penalty_values:
    rss, nzs= model_l1_limit(l1_penalty=l1_pen)
    model_list2.append(rss)
    non_zeros.append(nzs)
    #list2_coeffs.append(coefficients)

L1 2976351441.6313133 : [196100.93780562267, 2181.574321070672, 17962.696661157832, 34.14246565122559, 789.319789078234, 3665.9308175965243, 11333.841030833135, 3578.9004004381122, 32.74320137175839, 12.795381135944734] 966925692362086.5
L1 3019316638.9524155 : [198563.24621808337, 2067.0151555582247, 17810.387597798555, 34.021510305615955, 782.1823176948214, 3358.2033052237985, 9876.737608119618, 3528.25500887085, 32.53723292119236, 11.050728857454825] 974019450084556.8
L1 3062281836.2735176 : [201025.56040541275, 1952.476489609668, 17658.089496535533, 33.90056535462458, 775.0454285495282, 3050.472194038408, 8418.987059021993, 3477.612166173274, 32.33126559851514, 9.306267213386935] 981188367942450.6
L1 3105247033.5946198 : [203443.60100002843, 1825.7886426921725, 17487.49235108723, 33.76199379582763, 766.9631650719977, 2716.8076217860025, 6944.3661608031025, 3421.3503525231645, 32.10532315753412, 7.513151240077799] 989328342459471.6
L1 3148212230.915722 : [205805.89950145458, 1683.39

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [24]:
l1_penalty_min = np.max(l1_penalty_values[[i for i in range(len(non_zeros)) if non_zeros[i] == max_nonzeros]])
l1_penalty_min

3577864204.126743

In [25]:
for i in list(l1_penalty_values[[i for i in range(len(non_zeros)) if non_zeros[i] == max_nonzeros]]):
    print(model_list2[list(l1_penalty_values).index(i)])
list(l1_penalty_values[[i for i in range(len(non_zeros)) if non_zeros[i] == max_nonzeros]])

1046937488751713.5
1051147625612863.0
1055992735342998.9
1060799531763289.9


[3448968612.163437, 3491933809.484539, 3534899006.805641, 3577864204.126743]

In [26]:
model_all2 = turicreate.linear_regression.create(sales, target='price', features=all_features,
                                                validation_set=None, 
                                                l2_penalty=0., l1_penalty=3448968612.163437)
model_all2.coefficients[model_all.coefficients['value']!=0][:]

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 10        | 11       | 0.000002  | 0.437038     | 5986314.583941     | 293985.646175                   |

+-----------+----------+-----------+--------------+--------------------+---------------------------------+

Completed (Iteration limit reached).

This model may not be optimal. To improve it, consider increasing `max_iterations`.

name,index,value,stderr
(intercept),None,113003.25561650672,None
bathrooms,None,21448.4723121285,None
sqft_living,None,35.720260923973655,None
sqft_living_sqrt,None,969.5527902084325,None
grade,None,5022.01290720405,None
sqft_above,None,36.73030536670435,None
